In [1]:
import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
# Problem parameters
np.random.seed(42)  # For reproducibility

def create_problem(size):
    
    B=np.random.randint(80,90)
    if size=='small':
        B=np.random.randint(20,30)
    block=np.zeros((6,B))
    start_position=np.random.choice(range(42),B)
    target_position=np.random.choice(range(42),B)
    ready_times = np.random.uniform(0, 480, B)
    due_times = np.random.uniform(ready_times + 60, 600, B)
    weights=np.random.uniform(10, 100, B)
    loading_unloading_times = np.random.uniform(5, 15) + 0.05 * weights
    block[0]=start_position.astype('int64')
    block[1]=target_position.astype('int64')
    block[2]=ready_times
    block[3]=due_times
    block[4]=weights
    block[5]=loading_unloading_times
    #start position
    #target position
    #ready_time
    #due_dates
    #weights
    #loading_unloading_times
    
    T = np.random.randint(12,15)  # Number of transporters
    if size=='small':
        T=np.random.randint(3,4)
    TT = np.random.choice([1, 2, 3], T) # Number of transporter types
    transporter=np.zeros((5,T))   
    transporter[0]=TT                      # transporter type
    transporter[1]=25+TT*25                #maximum deadweight 
    transporter[2]=375-75*transporter[0]   #  speed
    while np.max(transporter[1])<np.max(weights):
        TT = np.random.choice([1, 2, 3], T)
        transporter[0]=TT
        transporter[1]=25+TT*25
        transporter[2]=375-75*transporter[0]
        
    transporter[3]=-1
    transporter[4]=0
    # Distances
    distances = np.random.uniform(50, 5000, size=(42, 42))
    symmetric_matrix = (distances + distances.T) / 2
    np.fill_diagonal(symmetric_matrix, 0)
    distances=symmetric_matrix
    transporter_initial_position=np.random.choice(range(42),T)
 
    return B,T,transporter, block, distances,transporter_initial_position

In [3]:
create_problem('small')

(23,
 3,
 array([[  2.,   3.,   2.],
        [ 75., 100.,  75.],
        [225., 150., 225.],
        [ -1.,  -1.,  -1.],
        [  0.,   0.,   0.]]),
 array([[ 28.        ,  14.        ,   7.        ,  20.        ,
          38.        ,  18.        ,  22.        ,  10.        ,
          10.        ,  23.        ,  35.        ,  39.        ,
          23.        ,   2.        ,  21.        ,   1.        ,
          23.        ,  29.        ,  37.        ,   1.        ,
          20.        ,  32.        ,  11.        ],
        [ 21.        ,  24.        ,  26.        ,  41.        ,
          27.        ,  15.        ,  14.        ,   2.        ,
          36.        ,   6.        ,  20.        ,   8.        ,
          38.        ,  17.        ,   3.        ,  24.        ,
          13.        ,   8.        ,  25.        ,   1.        ,
          19.        ,  27.        ,   6.        ],
        [292.79839574, 399.93355763,  83.21503368, 187.70909164,
          87.47332214, 362.573

In [3]:
B=30
T=6
transporter=np.array([[  1,   1,   1, 3, 3, 3],
        [ 50., 50.,  50., 100, 100, 100],
        [120, 120., 120.,120,120,120],
        [ -1,-1,-1,-1,-1,-1],
        [  0.,   0.,   0,0,0,0]])


In [44]:
distance=pd.read_excel('validation.xlsx',index_col=0,sheet_name='dis')
block_case=[]
for i in range(20):
    sname='block'+str(i)
    case_study=np.array(pd.read_excel('validation.xlsx',index_col=0,sheet_name=sname)).T
    block=[]
    block.append(case_study[0])
    block.append(case_study[1])
    block.append(case_study[3]*300)
    block.append(case_study[4]*300)
    block.append(case_study[6]*50+25)
    block.append(case_study[6]*0)
    block=np.array(block)
    block_case.append(block)

## ACO_RS

In [5]:


def cal_t(i,k):
    i=int(i)
    k=int(k)
    return block[5][i]+(distance[int(block[0][i])][int(block[1][i])])/transporter[2][k]*3/2+block[5][i]

def cal_e(i,j,k):
    i=int(i)
    j=int(j)
    k=int(k)
    return (distance[int(block[1][i])][int(block[0][j])])/transporter[2][k]

def visibility(x,i,j,k):
    i=int(i)
    j=int(j)
    k=int(k)
    return 1/(max(block[2][j],x+cal_e(i,j,k))+cal_t(j,k))

def select_agent(event):
    argmin=np.where(event.min()==event)
    agent=random.choice(argmin[0])
    return agent

def select_target(unvisited,block,transporter,agent,valid,mode,pheromone):
    block_info=block.copy()
    q_m=0.3
    q_z=0.05
    alpha=0.8
    beta=0.2
    possible_selection=np.where(block_info[4]<transporter[1][agent])[0]
    possible_selection=np.intersect1d(possible_selection, unvisited)

    value=np.array([])
    if len(possible_selection)==0:
        return 0,True
    for j in possible_selection:
        if transporter[3][agent]==-1:
            
            temp_value=(1/(block[2][int(j)]+cal_t(j,agent)))**beta+pheromone[0][int(j)]**alpha
        else:
            temp_value=visibility(transporter[4][int(agent)],transporter[3][int(agent)],j,agent)**beta
            +pheromone[int(transporter[3][agent])][int(j)]**alpha
        value=np.append(value,temp_value)
    q=random.uniform(0,1)
    if valid:
        q=0
    if q >q_m:
        x=value
        f_x = x / np.sum(x)
        select=np.random.choice(possible_selection,p=f_x)
    elif (q>q_m-q_z) & (mode=='ACO_RS'):
        select=np.random.choice(possible_selection)    
    else:
        select=np.argmax(value)
        select=possible_selection[select]
    
    return select,False


def simulation(B,T,transporter,block,pheromone,valid,mode):
    unvisited_set=np.array(range(B))
    empty_time=0
    waiting_time=0
    tardy_time=0
    transporter[4]=0
    transporter[3]=-1
    update=np.zeros((B,B))


    while len(unvisited_set)>0:
        agent=select_agent(transporter[4])

        select,done=select_target(unvisited_set,block,transporter,agent,valid,mode,pheromone)
        start_time=transporter[4][agent]
        
        if done:
            transporter[4][agent]=np.inf
            continue
        update[int(transporter[3][agent])][int(select)]=1
        if transporter[3][int(agent)]==-1:
            empty_time+=(distance[0][int(block[0][select])])/transporter[2][agent]

            transporter[4][int(agent)]=block[2][int(select)]+cal_t(select,agent)

        else:
            empty_time+=cal_e(transporter[3][int(agent)],select,agent)
            waiting_time+=max(transporter[4][int(agent)]+cal_e(transporter[3][int(agent)],select,agent)-block[2][int(select)],0)
            transporter[4][int(agent)]=max(block[2][int(select)],transporter[4][int(agent)]+cal_e(transporter[3][int(agent)],
            select,agent))+cal_t(select,agent)
        transporter[3][int(agent)]=select
        tardy_time+=max(0,transporter[4][int(agent)]-block[3][int(select)])
        unvisited_set=np.delete(unvisited_set,np.where(unvisited_set==select)[0])

    return empty_time,waiting_time,tardy_time,update


def run(B,T,transporter,block,distance,iteration,We,Ww,Wd,mode,valid_step):
    history=[]
    validation=[]
    compute_time=[]
    
    pheromone=np.ones((B,B))/B
    next_pheromone=pheromone.copy()
    s_time=time.time()
    all_time_best=0
    for ite in range(1,iteration+1):
        best_z=0
        if ite%10==0:
            print(ite)
        for i in range(2*B):

            empty_time,waiting_time,tardy_time,update=simulation(B,T,transporter,block,pheromone,False,mode)

            history.append(We*empty_time+Ww*waiting_time+Wd*tardy_time)
            
            z=1/(We*empty_time+Ww*waiting_time+Wd*tardy_time)
            if z>best_z:
                best_z=z
                best_update=update
            next_pheromone=update_pheromone(next_pheromone,update,0.05,z)
        if best_z>all_time_best:
            all_time_best=best_z
        next_pheromone=update_pheromone(next_pheromone,best_update,0.1,best_z)
        pheromone=next_pheromone
        
    return history,validation,compute_time,pheromone,1/all_time_best

def update_pheromone(pheromone,update,evaporate,z):

    pheromone=(1-evaporate)*pheromone+update*z
    return pheromone


In [3]:
B=100
T=10

distance=pd.read_excel('validation.xlsx',index_col=0,sheet_name='dis')
block_case=[]
for i in range(20):
    sname='block'+str(i)
    case_study=np.array(pd.read_excel('validation.xlsx',index_col=0,sheet_name=sname)).T
    block=[]
    block.append(case_study[0])
    block.append(case_study[1])
    block.append(case_study[3]*600)
    block.append(case_study[4]*600)
    block.append(case_study[6]*50+25)
    block.append(case_study[6]*0)
    block=np.array(block)
    block_case.append(block)

In [ ]:
total_validation=[]
total_compute_time=[]
for i in range(20):
    B=100
    T=10
    transporter=np.array([[  1,   1,   1,1,1, 3, 3, 3,3,3],
        [ 50., 50.,  50.,50,50, 100, 100, 100,100,100],
        [120, 120., 120.,120,120,120,120,120,120,120],
        [ -1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        [  0.,   0.,   0,0,0,0,   0,0,0,0]])
    block=block_case[i]
    history,validation,compute_time,pheromone,rpd_best=run(B,T,transporter,block,distance,100,1,0,1,'ACO_RS',100)
    total_validation.append(validation)
    total_compute_time.append(np.array(compute_time).mean())
   
    print(rpd_best)
    
    

10
20
30
40
50


In [ ]:
total_validation=[]
total_compute_time=[]
for i in range(20):
    B=100
    T=10
    transporter=np.array([[  1,   1,   1,1,1, 3, 3, 3,3,3],
        [ 50., 50.,  50.,50,50, 100, 100, 100,100,100],
        [120, 120., 120.,120,120,120,120,120,120,120],
        [ -1,-1,-1,-1,-1,-1,-1,-1,-1,-1],
        [  0.,   0.,   0,0,0,0,   0,0,0,0]])
    block=block_case[i]
    history,validation,compute_time,pheromone,rpd_best=run(B,T,transporter,block,distance,1000,1,0,1,'ACO',100)
    total_validation.append(validation)
    total_compute_time.append(np.array(compute_time).mean())
   
    print(rpd_best)
    
    

## Genetic Alogrithm

In [ ]:
B=30
T=6
transporter=np.array([[  1,   1,   1, 3, 3, 3],
        [ 50., 50.,  50., 100, 100, 100],
        [120, 120., 120.,120,120,120],
        [ -1,-1,-1,-1,-1,-1],
        [  0.,   0.,   0,0,0,0]])
distance=pd.read_excel('validation.xlsx',index_col=0,sheet_name='dis')
block_case=[]
for i in range(20):
    sname='block'+str(i)
    case_study=np.array(pd.read_excel('validation.xlsx',index_col=0,sheet_name=sname)).T
    block=[]
    block.append(case_study[0])
    block.append(case_study[1])
    block.append(case_study[3]*300)
    block.append(case_study[4]*300)
    block.append(case_study[6]*50+25)
    block.append(case_study[6]*0)
    block=np.array(block)
    block_case.append(block)

In [7]:

def select_target_for_P2(unvisited,current,distance,pheromone,block,mode):
    alpha=1
    beta=1
    value=np.array([])
    current=int(current)
    q_m=0.3
    q_z=0.05
    
    for j in unvisited:
        j=int(j)
        temp_value=(1/(distance[int(block[1][current])][int(block[0][j])]+10))**beta+pheromone[current][int(j)]**alpha
        value=np.append(value,temp_value)
    q=random.uniform(0,1)
    if q >q_m+q_z:
        x=value
        
        f_x = x / np.sum(x)
        select=np.random.choice(unvisited,p=f_x)
    elif (q>q_m) & (mode=='ACO_RS'):
        select=np.random.choice(unvisited)    
    else:
        select=np.argmax(value)
        select=unvisited[select]
    
    return select,distance[int(block[1][current])][int(block[0][select])]


def ACO_for_P2(iteration,Q,distance,B,block):
    pheromone=np.ones((B,B))/B
    next_pheromone=pheromone.copy()
    
    for ite in range(1,iteration+1):
        best_z=0
        for i in range(2*B):
            current=int(i%B)
            update=np.zeros((B,B))
            sum=0
            
            unvisited=np.array(range(B))
            while len(unvisited)>1:
                unvisited=np.delete(unvisited,np.where(unvisited==current)[0])
                next,z=select_target_for_P2(unvisited,current,distance,pheromone,block,mode='ACO_RS')
                update[current][next]=1
                sum+=z
                current=next
            
            next_pheromone=update_pheromone(next_pheromone,update,0.05,Q/sum)
            if Q/sum>best_z:
                best_z=Q/sum
                best_update=update
        next_pheromone=update_pheromone(next_pheromone,best_update,0.1,best_z)   
        pheromone=next_pheromone.copy()
        
    return pheromone

In [8]:


def cal_t(i,k):
    i=int(i)
    k=int(k)
    return block[5][i]+(distance[int(block[0][i])][int(block[1][i])])/transporter[2][k]*10/6+block[5][i]

def cal_e(i,j,k):
    i=int(i)
    j=int(j)
    k=int(k)
    return (distance[int(block[1][i])][int(block[0][j])])/transporter[2][k]


def select_first_job(distance,block,transporter_initial_position,B):
    list_of_first_job=[]
    candidate=np.array(range(B))
    for tp_start in transporter_initial_position:
        d=distance[int(tp_start)][block[0][candidate].astype('int32')]
        min_d=np.where(d.min()==d)
        selected_first_job=random.choice(min_d[0])
        list_of_first_job.append(candidate[selected_first_job])
        candidate=np.delete(candidate,selected_first_job)
    return list_of_first_job
    
def update_pheromone(pheromone,update,evaporate,z):
    pheromone=(1-evaporate)*pheromone+update*z
    return pheromone

def simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,sequence,nojfet,penalty):
    empty_time=0
    tardy_time=0
    weight_constraint=0
    #penalty: weight constraint, ready time
    transporter[4]=0
    transporter[3]=-1
    sequence=sequence.astype('int32')
    step=0
    for agent,j in enumerate(nojfet):
        current_time=0
        tp_current_position=int(transporter_initial_position[agent])
        for i in range(int(j)):
            empty_move_time=(distance[int(tp_current_position)][int(block[0][sequence[step]])])/transporter[2][int(agent)]    
            empty_time+=empty_move_time
            current_time+=empty_move_time
            tp_current_postion=block[0][sequence[step]]
            if i==0:
                current_time=0
            start_time=max(block[2][sequence[step]],current_time)
            end_time=start_time+cal_t(sequence[step],agent)
            weight_constraint+=max(block[4][sequence[step]]-transporter[1][agent],0)
            tardy_time+=max(0,end_time-block[3][sequence[step]])
            current_time=end_time
            tp_current_position=block[1][sequence[step]]
            step+=1
    fitness=empty_time+tardy_time+weight_constraint*penalty[0]
    
    return fitness,empty_time,tardy_time,weight_constraint*penalty[0]
            
                


    

In [9]:
def assign_policy(B,T,transporter,block,distance,transporter_initial_position,pheromone):
    list_of_current_job=select_first_job(distance,block,transporter_initial_position,B)
    
    job_list=[]
    for i in list_of_current_job:
        job_list.append([i])
    unvisited=np.array(range(B))

    while len(unvisited)>1:
        unvisited=np.setdiff1d(unvisited,list_of_current_job)
        
        maximum=0
        next_action=[0,0]
        for ind,i in enumerate(list_of_current_job):
            if maximum<np.max(pheromone[i][unvisited]):
                maximum=np.max(pheromone[i][unvisited])
                next_action[0]=ind                        # index
                next_action[1]=unvisited[np.argmax(pheromone[i][unvisited])]    # next assignment
        job_list[next_action[0]].append(next_action[1])
        list_of_current_job[next_action[0]]=next_action[1]
    number_of_job_for_each_transporter=[]
    initial_solution=np.array([])
    for i in job_list:
        number_of_job_for_each_transporter.append(len(i))
        initial_solution=np.append(initial_solution,np.array(i))
    return number_of_job_for_each_transporter,initial_solution

In [164]:
number_of_job_for_each_transporter,initial_solution=assign_policy(B,T,transporter,block,distance,transporter_initial_position,pheromone_matric)
print(initial_solution)
print(number_of_job_for_each_transporter)

[14, 26, 20, 15]
[[14], [26], [20], [15]]
[14. 11.  6. 25.  7. 23. 19.  5.  9. 10.  4. 13. 28. 16. 26.  2. 22. 20.
 17. 12. 15.  0.  8. 27. 24.  3. 21. 18.  1. 29.]
[14, 3, 3, 10]


In [11]:

B=30
T=6
transporter=np.array([[  1,   1,   1, 3, 3, 3],
        [ 50., 50.,  50., 100, 100, 100],
        [120, 120., 120.,120,120,120],
        [ -1,-1,-1,-1,-1,-1],
        [  0.,   0.,   0,0,0,0]])
distance=pd.read_excel('validation.xlsx',index_col=0,sheet_name='dis')
block_case=[]
for i in range(20):
    transporter=np.array([[  1,   1,   1, 3, 3, 3],
        [ 50., 50.,  50., 100, 100, 100],
        [120, 120., 120.,120,120,120],
        [ -1,-1,-1,-1,-1,-1],
        [  0.,   0.,   0,0,0,0]])
    sname='block'+str(i)
    case_study=np.array(pd.read_excel('validation.xlsx',index_col=0,sheet_name=sname)).T
    block=[]
    block.append(case_study[0])
    block.append(case_study[1])
    block.append(case_study[3]*300)
    block.append(case_study[4]*300)
    block.append(case_study[6]*50+25)
    block.append(case_study[6]*0)
    block=np.array(block)
    
    transporter_initial_position=[0,0,0,0,0,0]
    
    Q=5000.0
    iteration=1000
    pheromone_matric=ACO_for_P2(iteration,Q,distance,B,block)
    number_of_job_for_each_transporter,initial_solution=assign_policy(B,T,transporter,block,distance,transporter_initial_position,pheromone_matric)
    print(initial_solution)
    print(number_of_job_for_each_transporter)
    # 목적 함수 (최소화하려는 함수)
    # fitness = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,sequence,nojfet,penalty=[20])
    
    # 초기화
    population_size = 50
    chromosome_length = B
    mutation_rate = 0.01
    generations = 10000
    
    # 초기 인구 생성
    population = np.zeros((population_size, chromosome_length))
    nojfet=np.zeros((population_size,T))
    for i in range(population_size):
        nojfet[i]=number_of_job_for_each_transporter
        sum=0
        for j in range(T):
            temp=int(nojfet[i][j])
            
            if temp!=0:
                population[i][sum:sum+temp]=np.random.choice(initial_solution[sum:sum+temp],temp,replace=False)
            sum+=temp
            # 메인 루프
    
    for generation in range(generations):
        # 적합도 평가
        fitness_list=np.zeros(population_size)
        for i in range(population_size):
            fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[i],nojfet[i],penalty=[100])
            fitness_list[i]=1/fitness
        if generation%1000==0:
            best_solution = population[np.argmax(fitness_list)]
            print(f"Generation {generation+1}: Best Solution - {best_solution}, Best Fitness - {1/np.max(fitness_list)}")
    
        
        # 토너먼트 선택
        x=fitness_list
        
        f_x = x / np.sum(x)
    
        selected_indices = np.random.choice(range(population_size), 2,p=f_x )
        parent = population[selected_indices]
        
        # 교차 (크로스오버)
        new_nojfet=np.zeros((population_size,T))
        child=-np.ones((population_size,chromosome_length))
        for j in range(population_size):
            selected_indices = np.random.choice(range(population_size), 2,p=f_x )
            parent = population[selected_indices]
            
            ratio=fitness_list[selected_indices[0]]/(fitness_list[selected_indices[0]]+fitness_list[selected_indices[1]])
            new_nojfet[j]=nojfet[selected_indices[0]].copy()
            if ratio<0.5:
                parent=parent[[1,0]]
                ratio=1-ratio
                new_nojfet[j]=nojfet[selected_indices[1]].copy()
            main_ind=np.random.choice(chromosome_length,int(chromosome_length*ratio)+1)
            left_over = [x for x in parent[1] if x not in parent[0][main_ind]]
            child[j][main_ind]=parent[0][main_ind]
            step=0
            for i in range(chromosome_length):
                if child[j][i]<0:
                    child[j][i]=left_over[step]
                    step+=1
            mu=random.uniform(0,1)
            if mu<mutation_rate:
                tp_c=np.random.choice(range(T),2)
                while new_nojfet[j][tp_c[0]]==0:
                    tp_c=np.random.choice(range(T),2)
                sum=0
                for num in range(tp_c[0]):
                    sum+=new_nojfet[j][num]
                mutation=random.randint(sum,sum+new_nojfet[j][tp_c[0]]-1)
                sum=0
                for num in range(tp_c[1]):
                    sum+=new_nojfet[j][num]
                
                insert_index=random.randint(sum,sum+new_nojfet[j][tp_c[1]])-1
                temp=child[j][mutation]
                new_child=child[j].copy()
                new_child=np.delete(new_child,mutation)
                new_child=np.insert(new_child,insert_index,temp)
                child[j]=new_child.copy()
                new_nojfet[j][tp_c[0]]-=1
                new_nojfet[j][tp_c[1]]+=1
            
        nojfet=new_nojfet.copy()
        population=child.copy()
    
        # 현재 세대에서의 최적 해 출력
    for i in range(population_size):
            fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[i],nojfet[i],penalty=[100])
            fitness_list[i]=1/fitness
        
    best_solution = population[np.argmax(fitness_list)]
    print(f"Generation {10000}: Best Solution - {best_solution}, Best Fitness - {1/np.max(fitness_list)}")
    fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[np.argmax(fitness_list)],nojfet[np.argmax(fitness_list)],penalty=[100])
    print(fitness,e,t,w)

[ 0.  8. 25.  4. 22.  1. 24.  7. 16. 19.  6. 23. 12.  2. 29.  5. 14. 26.
 28.  9. 21. 11. 17.  3. 27. 13. 10. 15. 20. 18.]
[3, 11, 4, 1, 7, 4]
Generation 1: Best Solution - [ 0. 25.  8. 12.  6. 22.  4. 24. 16. 19.  2.  1. 23.  7. 29.  5. 14. 26.
 28.  9.  3. 17. 13. 21. 11. 27. 10. 20. 18. 15.], Best Fitness - 21169.690982476997
Generation 1001: Best Solution - [ 8. 29. 27. 26. 24. 12.  6.  1. 19. 28.  5.  0.  3. 11. 13. 15. 23.  9.
 20. 17. 25. 10.  4. 22.  2. 14. 21. 16.  7. 18.], Best Fitness - 387.04260281964406
Generation 2001: Best Solution - [ 8. 29. 27. 26. 24. 12.  6.  1. 19. 28.  5.  3. 11. 13.  0. 15. 23.  9.
 20. 17. 25. 10.  4. 22.  2. 14. 21. 16.  7. 18.], Best Fitness - 376.08296944858245
Generation 3001: Best Solution - [ 8. 29. 27. 26. 24. 12.  6.  1. 19. 28.  5.  3. 11. 13.  0. 15. 23.  9.
 20. 17. 25. 10.  4. 22.  2. 14. 21. 16.  7. 18.], Best Fitness - 376.08296944858245
Generation 4001: Best Solution - [ 8. 29. 27. 26. 24. 12.  6.  1. 19. 28.  5.  3. 11. 13.  0. 15

In [260]:
history,validation,compute_time,pheromone,rpd_best=run(B,T,transporter,block,distance,1000,1,0,1,'ACO_RS',100,transporter_initial_position)
print(np.array(history).mean())
print(rpd_best)
print(validation[-1])
print(validation[-1]/rpd_best-1)

7275.463984893193
1019.7565960778577
1264.0827892080601
0.23959265776747007


In [263]:
B,T,transporter,block,distance,transporter_initial_position=create_problem('small')
s_time=time.time()
Q=5000.0
iteration=100
print(B,T)
pheromone_matric=ACO_for_GA(iteration,Q,distance,B)
number_of_job_for_each_transporter,initial_solution=assign_policy(B,T,transporter,block,distance,transporter_initial_position,pheromone_matric)
print(initial_solution)
print(number_of_job_for_each_transporter)
# 목적 함수 (최소화하려는 함수)
# fitness = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,sequence,nojfet,penalty=[20])

# 초기화
population_size = 50
chromosome_length = B
mutation_rate = 0.01
generations = 10000

# 초기 인구 생성
population = np.zeros((population_size, chromosome_length))
nojfet=np.zeros((population_size,T))
for i in range(population_size):
    nojfet[i]=number_of_job_for_each_transporter
    sum=0
    for j in range(T):
        temp=int(nojfet[i][j])
        
        if temp!=0:
            population[i][sum:sum+temp]=np.random.choice(initial_solution[sum:sum+temp],temp,replace=False)
        sum+=temp
        # 메인 루프

for generation in range(generations):
    # 적합도 평가
    fitness_list=np.zeros(population_size)
    for i in range(population_size):
        fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[i],nojfet[i],penalty=[100])
        fitness_list[i]=1/fitness
    if generation%1000==0:
        best_solution = population[np.argmax(fitness_list)]
        print(f"Generation {generation+1}: Best Solution - {best_solution}, Best Fitness - {1/np.max(fitness_list)}")

    
    # 토너먼트 선택
    x=fitness_list
    
    f_x = x / np.sum(x)

    selected_indices = np.random.choice(range(population_size), 2,p=f_x )
    parent = population[selected_indices]
    
    # 교차 (크로스오버)
    new_nojfet=np.zeros((population_size,T))
    child=-np.ones((population_size,chromosome_length))
    for j in range(population_size):
        selected_indices = np.random.choice(range(population_size), 2,p=f_x )
        parent = population[selected_indices]
        
        ratio=fitness_list[selected_indices[0]]/(fitness_list[selected_indices[0]]+fitness_list[selected_indices[1]])
        new_nojfet[j]=nojfet[selected_indices[0]].copy()
        if ratio<0.5:
            parent=parent[[1,0]]
            ratio=1-ratio
            new_nojfet[j]=nojfet[selected_indices[1]].copy()
        main_ind=np.random.choice(chromosome_length,int(chromosome_length*ratio)+1)
        left_over = [x for x in parent[1] if x not in parent[0][main_ind]]
        child[j][main_ind]=parent[0][main_ind]
        step=0
        for i in range(chromosome_length):
            if child[j][i]<0:
                child[j][i]=left_over[step]
                step+=1
        mu=random.uniform(0,1)
        if mu<mutation_rate:
            tp_c=np.random.choice(range(T),2)
            while new_nojfet[j][tp_c[0]]==0:
                tp_c=np.random.choice(range(T),2)
            sum=0
            for num in range(tp_c[0]):
                sum+=new_nojfet[j][num]
            mutation=random.randint(sum,sum+new_nojfet[j][tp_c[0]]-1)
            sum=0
            for num in range(tp_c[1]):
                sum+=new_nojfet[j][num]
            
            insert_index=random.randint(sum,sum+new_nojfet[j][tp_c[1]])-1
            temp=child[j][mutation]
            new_child=child[j].copy()
            new_child=np.delete(new_child,mutation)
            new_child=np.insert(new_child,insert_index,temp)
            child[j]=new_child.copy()
            new_nojfet[j][tp_c[0]]-=1
            new_nojfet[j][tp_c[1]]+=1
        
    nojfet=new_nojfet.copy()
    population=child.copy()

    # 현재 세대에서의 최적 해 출력
for i in range(population_size):
        fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[i],nojfet[i],penalty=[100])
        fitness_list[i]=1/fitness
    
best_solution = population[np.argmax(fitness_list)]
print(f"Generation {10000}: Best Solution - {best_solution}, Best Fitness - {1/np.max(fitness_list)}")
fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[np.argmax(fitness_list)],nojfet[np.argmax(fitness_list)],penalty=[100])
print(fitness,e,t,w)
print((time.time()-s_time)/60)

23 3
[ 2. 11.  9. 22.  0. 14. 18.  5. 19. 15.  7. 20. 16. 13.  6.  1. 21.  4.
 10.  3. 17.  8. 12.]
[17, 2, 4]
Generation 1: Best Solution - [ 9.  0.  7. 21.  6.  1. 15. 14.  5. 20. 19.  2. 18. 22. 16. 11. 13.  4.
 10. 17. 12.  8.  3.], Best Fitness - 9735.596870563193
Generation 1001: Best Solution - [ 2.  7.  6. 11.  0. 20. 19.  1. 18. 13. 16.  4. 15. 21.  5. 22. 14.  9.
  3. 12. 17.  8. 10.], Best Fitness - 367.80299047418265
Generation 2001: Best Solution - [ 2.  7.  6. 11.  0. 20. 19.  1. 18. 13. 16.  4. 15. 21.  5. 22. 14.  9.
  3. 12. 17.  8. 10.], Best Fitness - 367.80299047418265
Generation 3001: Best Solution - [ 2.  7.  6. 11.  0. 20. 19.  1. 18. 16.  4. 15. 21.  5. 22. 14.  9.  3.
 17.  8. 12. 10. 13.], Best Fitness - 270.60582601411494
Generation 4001: Best Solution - [ 2.  7.  6. 11.  0. 20. 19.  1. 18. 16.  4. 15. 21.  5. 22. 14.  9.  3.
 17.  8. 12. 10. 13.], Best Fitness - 270.60582601411494
Generation 5001: Best Solution - [ 2.  7.  6. 11.  0. 20. 19.  1. 16.  4. 15. 

KeyboardInterrupt: 

In [264]:
for i in range(population_size):
        fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[i],nojfet[i],penalty=[100])
        fitness_list[i]=1/fitness
    
best_solution = population[np.argmax(fitness_list)]
print(f"Generation {10000}: Best Solution - {best_solution}, Best Fitness - {1/np.max(fitness_list)}")
fitness,e,t,w = simulation_for_GA(B,T,transporter,block,distance,transporter_initial_position,population[np.argmax(fitness_list)],nojfet[np.argmax(fitness_list)],penalty=[100])
print(fitness,e,t,w)
print((time.time()-s_time)/60)

Generation 10000: Best Solution - [ 2.  7.  6. 11.  0. 20. 19.  1. 16.  4. 15. 21.  5. 22. 14.  9.  3. 17.
  8. 12. 10. 18. 13.], Best Fitness - 252.20955059831715
252.20955059831718 246.54357946152962 5.665971136787562 0
2.319668451944987


In [265]:
history,validation,compute_time,pheromone,rpd_best=run(B,T,transporter,block,distance,1000,1,0,1,'ACO_RS',100,transporter_initial_position)
print(np.array(history).mean())
print(rpd_best)
print(validation[-1])
print(validation[-1]/rpd_best-1)

2883.3476687735806
363.68589042548473
777.6102081471998
1.138136861007875
